# T3. dataloader 的内部结构和基本使用

&emsp; 1 &ensp; 
 
&emsp; &emsp; 1.1 &ensp; 

&emsp; &emsp; 1.2 &ensp; 

&emsp; 2 &ensp; 

&emsp; &emsp; 2.1 &ensp; 

&emsp; &emsp; 2.2 &ensp; 

&emsp; 3 &ensp; 
 
&emsp; &emsp; 3.1 &ensp; 

&emsp; &emsp; 3.2 &ensp; 

## 3. fastNLP 中的 dataloader

### 3.1 collator 的概念与使用

在`fastNLP 0.8`中，在数据加载模块`DataLoader`之前，还存在其他的一些模块，负责例如对文本数据

&emsp; 进行补零对齐，即 **核对器`collator`模块**，进行分词标注，即 **分词器`tokenizer`模块**

&emsp; 本节将对`fastNLP`中的核对器`collator`等展开介绍，分词器`tokenizer`将在下一节中详细介绍

在`fastNLP 0.8`中，**核对器`collator`模块负责文本序列的补零对齐**，通过

In [ ]:
from fastNLP import Collator

collator = Collator()
# collator.set_pad(field_name='text', pad_val='<pad>')

&emsp; 

### 3.2 dataloader 的结构与使用

In [ ]:
from fastNLP import prepare_torch_dataloader

dl_bundle = prepare_torch_dataloader(data_bundle, train_batch_size=2)

print(type(dl_bundle), type(dl_bundle['train']))

In [ ]:
dataloader = prepare_torch_dataloader(datasets['train'], train_batch_size=2)
print(type(dataloader))
print(dir(dataloader))

In [ ]:
dataloader.collate_fn

In [ ]:
dataloader.batch_sampler

### 3.3 实例：NG20 的加载预处理

在`fastNLP 0.8`中，**`Trainer`模块和`Evaluator`模块分别表示“训练器”和“评测器”**

&emsp; 对应于之前的`fastNLP`版本中的`Trainer`模块和`Tester`模块，其定义方法如下所示

在`fastNLP 0.8`中，需要注意，在同个`python`脚本中先使用`Trainer`训练，然后使用`Evaluator`评测

&emsp; 非常关键的问题在于**如何正确设置二者的`driver`**。这就引入了另一个问题：什么是 `driver`？

In [ ]:
import pandas as pd

from fastNLP import DataSet
from fastNLP import Vocabulary

dataset = DataSet.from_pandas(pd.read_csv('./data/ng20_test.csv'))

In [ ]:
from functools import partial

encode = partial(tokenizer.encode_plus, max_length=100, truncation=True,
                 return_attention_mask=True)
# 会新增 input_ids 、 attention_mask 和 token_type_ids 这三个 field
dataset.apply_field_more(encode, field_name='text')

In [ ]:
target_vocab = Vocabulary(padding=None, unknown=None)

target_vocab.from_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='label')
target_vocab.index_dataset(*[ds for _, ds in data_bundle.iter_datasets()], field_name='label',
                           new_field_name='labels')
# 需要将 input_ids 的 pad 值设置为 tokenizer 的 pad 值
dataset.set_pad('input_ids', pad_val=tokenizer.pad_token_id)
dataset.set_ignore('label', 'text')  # 因为 label 是原始的不需要的 str ，所以我们可以忽略它，让它不要在 batch 的输出中出现